In [23]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from skimage import io

In [3]:
# class CatsAndDogsDataset(Dataset):
#     def __init__(self, csv_file, root_dir, transform=None):
#         self.annotations = pd.read_csv(csv_file)
#         self.root_dir = root_dir
#         self.transform = transform

#     def __len__(self):
#         return len(self.annotations) #25000 images, 50% cat, 50% dog

#     def __getitem__(self, index):
#         img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
#         image = io.imread(img_path)
#         y_label = torch.tensor(int(self.annotations.iloc[index, 1]))

#         if self.transform:
#             image = self.transform(image)

#         return (image, y_label)

In [1]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms

class DogsAndCatsDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        """
        Custom dataset cho ảnh chó và mèo.

        Args:
            root_dir (str): Thư mục gốc chứa các thư mục 'cats' và 'dogs'.
            transform (callable, optional): Biến đổi ảnh.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []

        for label, class_name in enumerate(['cats', 'dogs']):
            class_folder = os.path.join(root_dir, class_name) #root_folder/cats, root_folder/dogs
            if not os.path.isdir(class_folder):
                continue
            for filename in os.listdir(class_folder):
                if filename.lower().endswith(('.jpg', '.png', '.jpeg')):
                    path = os.path.join(class_folder, filename)
                    self.samples.append((path, label)) #danh sách gồm các tuple chứa đường dẫn đến với mỗi ảnh và nhãn của nó

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_path, label = self.samples[idx]
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

C:\Users\ASUS\AppData\Roaming\Python\Python312\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [3]:
# Các phép biến đổi ảnh
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])
])

In [13]:
# Tạo dataset và dataloader
dataset = DogsAndCatsDataset(root_dir='dataset/cats_and_dogs_filtered/train/', transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=True)

In [15]:
iterator = iter(dataloader)
first_batch = next(iterator)
first_batch

[tensor([[[[ 0.6235,  0.6235,  0.6314,  ..., -0.4980, -0.5216, -0.5529],
           [ 0.6392,  0.6392,  0.6471,  ..., -0.5059, -0.5059, -0.5059],
           [ 0.6471,  0.6549,  0.6549,  ..., -0.4980, -0.5059, -0.4902],
           ...,
           [ 0.0353,  0.0431,  0.0510,  ...,  0.4039,  0.3804,  0.3569],
           [ 0.0196,  0.0510,  0.0588,  ...,  0.4118,  0.3490,  0.3725],
           [-0.0118,  0.0353,  0.0510,  ...,  0.3961,  0.3490,  0.3882]],
 
          [[ 0.1608,  0.1765,  0.2000,  ..., -0.6235, -0.6471, -0.6863],
           [ 0.2000,  0.2157,  0.2471,  ..., -0.6314, -0.6314, -0.6392],
           [ 0.2549,  0.2784,  0.2941,  ..., -0.6157, -0.6314, -0.6471],
           ...,
           [-0.0275, -0.0196, -0.0118,  ...,  0.3490,  0.3176,  0.2941],
           [-0.0431, -0.0118, -0.0039,  ...,  0.3412,  0.2784,  0.3020],
           [-0.0745, -0.0275, -0.0118,  ...,  0.3020,  0.2627,  0.3176]],
 
          [[-0.1294, -0.1059, -0.0667,  ..., -0.8824, -0.9059, -0.9137],
           [-

In [21]:
data, label = first_batch[0], first_batch[1]
print(len(data))
print(len(label))

10
10


In [87]:
class_weight = []

In [89]:
for dirpath, dirnames, filenames in os.walk('dataset/cats_and_dogs_filtered/train/'):
    print(dirpath, dirnames, len(filenames)) #duyệt qua các folder và folder con và các file trong thư mục
    if(len(filenames)) > 0:
        class_weight.append(1/len(filenames))

dataset/cats_and_dogs_filtered/train/ ['cats', 'dogs'] 0
dataset/cats_and_dogs_filtered/train/cats [] 1000
dataset/cats_and_dogs_filtered/train/dogs [] 1000


In [91]:
class_weight

[0.001, 0.001]

In [97]:
for id, (data, label) in enumerate(dataloader):
    if id == 0:
        print(data, label)

tensor([[[[ 0.2314,  0.9373,  0.9137,  ...,  0.9765,  0.9765,  0.2706],
          [ 0.2471,  0.0353, -0.3020,  ...,  0.3804,  0.7333,  0.2392],
          [ 0.2157, -0.1922, -0.6157,  ...,  0.4824,  0.6314,  0.2627],
          ...,
          [ 0.2314, -0.2706, -0.8667,  ...,  0.3882,  0.5843,  0.2549],
          [ 0.2392,  0.6549,  0.5451,  ...,  0.8118,  0.8667,  0.2314],
          [-0.4196, -0.1216, -0.1451,  ..., -0.1137, -0.1137, -0.4196]],

         [[ 0.2392,  0.9451,  0.9294,  ...,  0.9529,  0.9608,  0.2706],
          [ 0.2471,  0.0353, -0.3020,  ...,  0.3176,  0.7020,  0.2314],
          [ 0.2078, -0.2000, -0.6078,  ...,  0.4039,  0.5765,  0.2471],
          ...,
          [ 0.2314, -0.2706, -0.8745,  ...,  0.3961,  0.5922,  0.2549],
          [ 0.2392,  0.6549,  0.5373,  ...,  0.8196,  0.8745,  0.2314],
          [-0.4196, -0.1294, -0.1529,  ..., -0.1137, -0.1137, -0.4196]],

         [[ 0.2157,  0.9765,  0.9922,  ...,  0.9059,  0.9294,  0.2471],
          [ 0.2471,  0.1686, -